In [1]:
# BERT
# Reference: https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_multi_label_classification.ipynb

In [2]:
# Drive dependencies
'''
from google.colab import drive
drive.mount('/content/drive')

import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/IN3045 NLP CW'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nGOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/IN3045 NLP CW'\nGOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)\nprint(os.listdir(GOOGLE_DRIVE_PATH))\n"

In [3]:
# Import modules

!pip install -q transformers

import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [4]:
# Select runtime device

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
# Load data

train_path = 'Data/train.csv'
test_path = 'Data/test.csv'
#train_path = '/content/drive/MyDrive/Colab Notebooks/IN3045 NLP CW/data/train.csv'
#test_path = '/content/drive/MyDrive/Colab Notebooks/IN3045 NLP CW/data/test.csv'

# Create original unmodified train and test dataframes for record
original_train_df = pd.read_csv(train_path)
original_test_df = pd.read_csv(test_path)

# Create train and test dataframes to be modified
train_df = original_train_df
test_df = original_test_df

# Define function to modify dataframe
def modify_df(df):

    # Create columns of numerical labels foreach category
    added_columns = ['Business', 'Sports', 'Politics', 'Health', 'Entertainment', 'Tech']

    # Set default value of numerical label columns to 0
    for col in added_columns:
        df[col] = 0

    # Transform string label to numerical label
    for i in range(len(df)):
        if df.loc[i, 'Category'] == 'business':
            df.loc[i, 'Business'] = 1
        elif df.loc[i, 'Category'] == 'sports':
            df.loc[i, 'Sports'] = 1
        elif df.loc[i, 'Category'] == 'politics':
            df.loc[i, 'Politics'] = 1
        elif df.loc[i, 'Category'] == 'health':
            df.loc[i, 'Health'] = 1
        elif df.loc[i, 'Category'] == 'entertainment':
            df.loc[i, 'Entertainment'] = 1
        else:
            df.loc[i, 'Tech'] = 1

    # Create column of numerical lists transformed from numerical label columns
    df['List'] = df[df.columns[3:]].values.tolist()

    # Drop unnecessary columns
    df = df.drop(['Excerpt', 'Category', 'Business', 'Sports', 'Politics', 'Health', 'Entertainment', 'Tech'], axis=1)

    return df

# Modify train and test dataframes
train_df = modify_df(train_df)
test_df = modify_df(test_df)

In [6]:
# Set hyperparameters and create tokeniser

MAX_LEN = 100
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/Users/anson/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# Define custom dataset

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.title = dataframe.Title
        self.targets = self.data.List
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [8]:
# Create train and test datasets and dataloaders

training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [9]:
# Create BERT model

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)

    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [10]:
# Train model (commented out and not used for testing)
'''
# Define loss function
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

# Create optimiser
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

# Define function to train
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%100==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Perform training
for epoch in range(EPOCHS):
    train(epoch)
'''

"\n# Define loss function\ndef loss_fn(outputs, targets):\n    return torch.nn.BCEWithLogitsLoss()(outputs, targets)\n\n# Create optimiser\noptimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)\n\n# Define function to train\ndef train(epoch):\n    model.train()\n    for _,data in enumerate(training_loader, 0):\n        ids = data['ids'].to(device, dtype = torch.long)\n        mask = data['mask'].to(device, dtype = torch.long)\n        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)\n        targets = data['targets'].to(device, dtype = torch.float)\n\n        outputs = model(ids, mask, token_type_ids)\n\n        optimizer.zero_grad()\n        loss = loss_fn(outputs, targets)\n        if _%100==0:\n            print(f'Epoch: {epoch}, Loss:  {loss.item()}')\n\n        optimizer.zero_grad()\n        loss.backward()\n        optimizer.step()\n\n# Perform training\nfor epoch in range(EPOCHS):\n    train(epoch)\n"

In [11]:
# Save / Load model for inference

#MODEL_PATH_AFTER_MYDRIVE = 'Colab Notebooks/IN3045 NLP CW/Models/bert.pth'
#MODEL_PATH = os.path.join('drive', 'My Drive', MODEL_PATH_AFTER_MYDRIVE)
#torch.save(model.state_dict(), MODEL_PATH)

MODEL_PATH = 'Model/bert.pth'
model_state_dict = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [12]:
# Validate / Test model

# Define function to validate
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

# Perform validation
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    
    print("Classification Report:")
    print(metrics.classification_report(targets, outputs))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/anson/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Accuracy Score = 0.8695652173913043
F1 Score (Micro) = 0.885766646304215
F1 Score (Macro) = 0.7813052490099163
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.97      0.87       224
           1       0.96      0.97      0.96       199
           2       0.95      0.88      0.92       190
           3       0.96      0.89      0.92        97
           4       0.90      0.84      0.87        67
           5       1.00      0.08      0.15        51

   micro avg       0.90      0.88      0.89       828
   macro avg       0.93      0.77      0.78       828
weighted avg       0.91      0.88      0.87       828
 samples avg       0.87      0.88      0.87       828



/Users/anson/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
